In [1]:
import math
import numpy as np
import wandb
import random
import torch
import torch_geometric
from torch_geometric.data import Data
import sys
import os
from tqdm import tqdm
import signal
import joblib
import argparse
import json
import os
import subprocess
from torch.utils.data import DataLoader, Dataset, Subset

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import help_functions as hf

scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
    
import gnn_io as gio
import gnn_architectures_district_features as garch
import copy

In [2]:
# Call this function during training without the scalars and with the directory path, and during the testing with the saved scalars and without a directory path to save.
def normalize_dataset(dataset_input, directory_path):
    dataset = copy_subset(dataset_input)
    dataset = normalize_x_values(dataset, directory_path)
    dataset = normalize_positional_features(dataset, directory_path)
    dataset = normalize_mode_stats(dataset, directory_path)
    return dataset
        
# Function to copy a Subset
def copy_subset(subset):
    return Subset(copy.deepcopy(subset.dataset), copy.deepcopy(subset.indices))

def normalize_x_values(dataset, directory_path):
    shape_of_x = dataset[0].x.shape[1]
    list_of_scalers_to_save = []
    x_values = torch.cat([data.x for data in dataset], dim=0)

    for i in range(shape_of_x):
        all_node_features = replace_invalid_values(x_values[:, i].reshape(-1, 1)).numpy()
        
        scaler = StandardScaler()
        print(f"Scaler created for x values at index {i}: {scaler}")
        scaler.fit(all_node_features)
        list_of_scalers_to_save.append(scaler)

        for data in dataset:
            data_x_dim = replace_invalid_values(data.x[:, i].reshape(-1, 1))
            normalized_x_dim = torch.tensor(scaler.transform(data_x_dim.numpy()), dtype=torch.float)
            if i == 0:
                data.normalized_x = normalized_x_dim
            else:
                data.normalized_x = torch.cat((data.normalized_x, normalized_x_dim), dim=1)

    joblib.dump(list_of_scalers_to_save, (directory_path + 'x_scaler.pkl'))
    for data in dataset:
        data.x = data.normalized_x
        del data.normalized_x
    return dataset

def normalize_positional_features(dataset, directory_path):
    # Initialize 6 StandardScalers for 3 sets of 2 dimensions
    scalers = [[StandardScaler() for _ in range(2)] for _ in range(3)]

    # Standardize the data
    for i in range(3):  # Iterate over the second dimension (3 sets)
        for j in range(2):  # Iterate over the third dimension (2D vectors)
            values = np.vstack([data.pos[:, i, j].numpy() for data in dataset]).reshape(-1, 1)
            # Fit the corresponding scaler on the extracted values
            scalers[i][j].fit(values)
            for data in dataset:
                transformed = scalers[i][j].transform(data.pos[:, i, j].numpy().reshape(-1, 1)).flatten()
                # Convert the transformed NumPy array back into a torch tensor
                data.pos[:, i, j] = torch.tensor(transformed, dtype=torch.float32)
    # Save the scalers using joblib
    for i in range(3):
        for j in range(2):
            # Dump the scalers with meaningful names to differentiate them
            scaler_path = directory_path + f'scaler_pos_{i}_{j}.pkl'
            joblib.dump(scalers[i][j], scaler_path)

    print("Postional scalers saved and dataset standardized.")
    return dataset

def normalize_mode_stats(dataset, directory_path):
    # Initialize 12 StandardScalers for 6 sets of 2 dimensions
    scalers = [[StandardScaler() for _ in range(2)] for _ in range(6)]

    # Standardize the data
    for i in range(6):  # Iterate over the first dimension (6 sets)
        for j in range(2):  # Iterate over the second dimension (2D vectors)
            values = np.vstack([data.mode_stats[i, j].numpy().reshape(-1, 1) for data in dataset])
            # Fit the corresponding scaler on the extracted values
            scalers[i][j].fit(values)
            for data in dataset:
                transformed = scalers[i][j].transform(data.mode_stats[i, j].numpy().reshape(-1, 1)).flatten()
                # Convert the transformed NumPy array back into a torch tensor
                data.mode_stats[i, j] = torch.tensor(transformed, dtype=torch.float32)
    
    # Save the scalers using joblib
    for i in range(6):
        for j in range(2):
            # Dump the scalers with meaningful names to differentiate them
            scaler_path = directory_path + f'scaler_mode_stats_{i}_{j}.pkl'
            joblib.dump(scalers[i][j], scaler_path)

    print("Mode stats scalers saved and dataset standardized.")
    return dataset

def replace_invalid_values(tensor):
    tensor[tensor != tensor] = 0  # replace NaNs with 0
    tensor[tensor == float('inf')] = 0  # replace inf with 0
    tensor[tensor == float('-inf')] = 0  # replace -inf with 0
    return tensor

In [3]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def prepare_data_with_graph_features(data_dict_list, batch_size, path_to_save_dataloader):
    datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y'], mode_stats=d['mode_stats']) for d in data_dict_list]
    train_set, valid_set, test_set = gio.split_into_subsets(dataset=datalist, train_ratio=0.8, val_ratio=0.15, test_ratio=0.05)
    
    train_set_normalized = normalize_dataset(dataset_input = train_set, directory_path=path_to_save_dataloader + "train_")
    valid_set_normalized = normalize_dataset(dataset_input = valid_set, directory_path=path_to_save_dataloader + "valid_")
    # test_set_normalized = normalize_dataset(dataset_input = test_set, directory_path=path_to_save_dataloader + "test_")
        
    train_loader = DataLoader(dataset=train_set_normalized, batch_size=batch_size, shuffle=True, num_workers=4, prefetch_factor=2, pin_memory=True, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    val_loader = DataLoader(dataset=valid_set_normalized, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    # test_loader = DataLoader(dataset=test_set_normalized, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=gio.collate_fn, worker_init_fn=seed_worker)
    # gio.save_dataloader(test_loader, path_to_save_dataloader + 'test_dl.pt')
    # gio.save_dataloader_params(test_loader, path_to_save_dataloader + 'test_loader_params.json')
    
    return train_loader, val_loader

In [4]:
# Set parameters here
params = {"project_name": "test",
            "num_epochs": 1000,
            "batch_size": 8,
            "point_net_conv_layer_structure_local_mlp": [64, 128],
            "point_net_conv_layer_structure_global_mlp": [256, 64],
            "gat_conv_layer_structure": [128, 256, 256, 128],
            "graph_mlp_layer_structure": [128, 256, 128],
            "lr": 0.001,
            "gradient_accumulation_steps": 3,
            "in_channels": 15,
            "out_channels": 1,
            "early_stopping_patience": 100,
            "unique_model_description": "my_test",
            "dropout": 0.3,
            "use_dropout": False
        } 

# Define the paths here
def get_paths(base_dir: str, unique_model_description: str, model_save_path: str = 'trained_model/model.pth'):
    data_path = os.path.join(base_dir, unique_model_description)
    os.makedirs(data_path, exist_ok=True)
    model_save_to = os.path.join(data_path, model_save_path)
    path_to_save_dataloader = os.path.join(data_path, 'data_created_during_training/')
    os.makedirs(os.path.dirname(model_save_to), exist_ok=True)
    os.makedirs(path_to_save_dataloader, exist_ok=True)
    return model_save_to, path_to_save_dataloader

def get_combined_data(dataset_path):
    data_dict_list = []
    batch_num = 1
    while True:
        try:
            batch_data = torch.load(os.path.join(dataset_path, f'datalist_batch_{batch_num}.pt'))
            data_dict_list.extend(batch_data)
            batch_num += 1
        except FileNotFoundError:
            break

    data_dict_list = torch.load(data_dict_list)
    return data_dict_list

In [5]:
# Create base directory for the run
base_dir = '../../data/' + params['project_name'] + '/'
unique_run_dir = os.path.join(base_dir, params['unique_model_description'])
os.makedirs(unique_run_dir, exist_ok=True)
dataset_path = '../../data/train_data/sim_output_1pm_capacity_reduction_10k_with_batches/'
data_dict_list = get_combined_data(dataset_path)

model_save_path, path_to_save_dataloader = get_paths(base_dir=base_dir, unique_model_description= params['unique_model_description'], model_save_path= 'trained_model/model.pth')

train_dl, valid_dl = prepare_data_with_graph_features(data_dict_list=data_dict_list, batch_size= params['batch_size'], path_to_save_dataloader= path_to_save_dataloader)

Total dataset length: 448
Training subset length: 358
Validation subset length: 67
Test subset length: 23
Scaler created for x values at index 0: StandardScaler()
Scaler created for x values at index 1: StandardScaler()
Scaler created for x values at index 2: StandardScaler()
Scaler created for x values at index 3: StandardScaler()
Scaler created for x values at index 4: StandardScaler()
Scaler created for x values at index 5: StandardScaler()
Scaler created for x values at index 6: StandardScaler()
Scaler created for x values at index 7: StandardScaler()
Scaler created for x values at index 8: StandardScaler()
Scaler created for x values at index 9: StandardScaler()
Scaler created for x values at index 10: StandardScaler()
Scaler created for x values at index 11: StandardScaler()
Scaler created for x values at index 12: StandardScaler()
Scaler created for x values at index 13: StandardScaler()
Scaler created for x values at index 14: StandardScaler()
Postional scalers saved and datase

In [6]:
gpus = hf.get_available_gpus()
best_gpu = hf.select_best_gpu(gpus)
hf.set_cuda_visible_device(best_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = hf.setup_wandb(params['project_name'], {
    "epochs": params['num_epochs'],
    "batch_size": params['batch_size'],
    "lr": params['lr'],
    "gradient_accumulation_steps": params['gradient_accumulation_steps'],
    "early_stopping_patience": params['early_stopping_patience'],
    "point_net_conv_local_mlp": params['point_net_conv_layer_structure_local_mlp'],
    "point_net_conv_global_mlp": params['point_net_conv_layer_structure_global_mlp'],
    "gat_conv_layer_structure": params['gat_conv_layer_structure'],
    "graph_mlp_layer_structure": params['graph_mlp_layer_structure'],
    "in_channels": params['in_channels'],
    "out_channels": params['out_channels'],
    "dropout": params['dropout'],
    "use_dropout": params['use_dropout']
})

model = garch.MyGnn(in_channels=config.in_channels, out_channels=config.out_channels, point_net_conv_layer_structure_local_mlp=config.point_net_conv_local_mlp,
                            point_net_conv_layer_structure_global_mlp=config.point_net_conv_global_mlp,
                            gat_conv_layer_structure=config.gat_conv_layer_structure,
                            graph_mlp_layer_structure=config.graph_mlp_layer_structure,
                            dropout=config.dropout, use_dropout=config.use_dropout)

model.to(device)

loss_fct = torch.nn.MSELoss()

baseline_loss_mean_target = gio.compute_baseline_of_mean_target(dataset=train_dl, loss_fct=loss_fct)
baseline_loss = gio.compute_baseline_of_no_policies(dataset=train_dl, loss_fct=loss_fct)
print("baseline loss mean " + str(baseline_loss_mean_target))
print("baseline loss no  " +str(baseline_loss) )

early_stopping = gio.EarlyStopping(patience=params['early_stopping_patience'], verbose=True)
best_val_loss, best_epoch = garch.train(model=model, 
            config=config, 
            loss_fct=loss_fct,
            optimizer=torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=1e-4),
            train_dl=train_dl,  
            valid_dl=valid_dl,
            device=device, 
            early_stopping=early_stopping,
            accumulation_steps=config.gradient_accumulation_steps,
            model_save_path=model_save_path,
            use_gradient_clipping=True,
            lr_scheduler_warmup_steps=20000,
            lr_scheduler_cosine_decay_rate=0.2)
print(f'Best model saved to {model_save_path} with validation loss: {best_val_loss} at epoch {best_epoch}')  

Using GPU 0 with CUDA_VISIBLE_DEVICES=0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


Model initialized
baseline loss mean 53.891822814941406
baseline loss no  54.452701568603516


Epoch 1/1000:   0%|          | 0/45 [00:00<?, ?it/s]/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/torch_geometric/warnings.py:11: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)
/opt/anaconda3/envs/chenhao-gnn/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([48, 2])) that is different to the input size (torch.Size([48, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/1000:   2%|▏         | 1/45 [00:01<00:46,  1.06s/it]

node_edge_loss: 55.73249053955078
graph_loss: 2.6652801036834717
node_edge_loss: 55.035804748535156
graph_loss: 0.6446346044540405
node_edge_loss: 56.49439239501953
graph_loss: 0.6178013682365417


...

node_edge_loss: 60.154815673828125
graph_loss: 19.128971099853516


Epoch 104/1000: 100%|██████████| 45/45 [00:08<00:00,  5.33it/s]


epoch: 103, validation loss: 71.72819052802191, lr: 0.00023395, r^2: 0.023187100887298584
EarlyStopping counter: 100 out of 100
Early stopping triggered. Stopping training.
Best validation loss:  54.62032981713613


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
lr,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
r^2,▁▂▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▄▅▆▆▆█
train_loss,▆▄▄█▄▇▅▆▆▅▅▆▄▅▃▄▄█▃▄▂▄▄▄▄▁▄▂▂▆▃▂▂▃▄▆▁▄▄▂
val_loss,▁▁▁▁▁▁▂▂▃▄▇▇▇▆██▇▇▇▇▇▇▆▇▆▆▇█▆▅▆▆▆▆▇▆▅▆▅▅
best_val_loss,54.62033
epoch,103
lr,0.00023
r^2,0.02319
train_loss,36.89248
val_loss,71.72819


Best model saved to ../../data/test/my_test/trained_model/model.pth with validation loss: 71.72819052802191 at epoch 103
